In [3]:
import numpy as np
import pandas as pd
%pip install keras
%pip install tensorflow
%pip install -U spacy
!python -m spacy download en_core_web_sm

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached keras-2.6.0-py2.py3-none-any.whl (1.3 MB)
Note: you may need to restart the kernel to use updated packages.
/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached tensorflow-2.6.0-cp37-cp37m-manylinux2010_x86_

In [7]:
movies_raw_df = pd.read_csv('s3://sagemaker-studio-716361152964-uifr8hhp3c/archive/wiki_movie_plots_deduped.csv')

movies_raw_df.head()

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...


In [8]:
movies_to_select = ((movies_raw_df['Genre'] == 'horror') &
                    # Restrict to American movies. 
                    (movies_raw_df['Origin/Ethnicity'] == 'American') &
                    # Only movies from 2000.
                    (movies_raw_df['Release Year'] > 2012))

In [9]:
horror_df = movies_raw_df[movies_to_select]['Plot']

horror_df.head()

16419    Five friends explore a supposedly haunted mine...
16429    Following a family dispute, Janet moves out of...
16442    Six high school seniors head out to a secluded...
16450    Sarah (Kate Bosworth) invites her childhood fr...
16469    Alone in her home, Margaret White, a disturbed...
Name: Plot, dtype: object

In [10]:
horror_df.shape

(61,)

In [11]:
horror_str = horror_df.str.cat(sep=' ')

In [12]:
import spacy

# Load language model. 
nlp = spacy.load('en_core_web_sm', disable = ['parser', 'tagger', 'ner', 'lemmatizer'])

In [18]:
def get_tokens(doc_text):
    # This pattern is a modification of the defaul filter from the
    # Tokenizer() object in keras.preprocessing.text. 
    # It just indicates which patters no skip.
    skip_pattern = '\r\n \n\n \n\n\n!"-#$%&()--.*+,-./:;<=>?@[\\]^_`{|}~\t\n\r '
    
    tokens = [token.text.lower() for token in nlp(doc_text) if token.text not in skip_pattern]
    
    return tokens

In [19]:
tokens = get_tokens(horror_str)

In [20]:
tokens[0:9]

['five',
 'friends',
 'explore',
 'a',
 'supposedly',
 'haunted',
 'mine',
 'to',
 'celebrate']

In [21]:
len(tokens)

38929

In [22]:
len_0 = 25

tokens[0:len_0]

['five',
 'friends',
 'explore',
 'a',
 'supposedly',
 'haunted',
 'mine',
 'to',
 'celebrate',
 'halloween',
 'exactly',
 'one',
 'hundred',
 'years',
 'after',
 'a',
 'family',
 'was',
 'murdered',
 'in',
 'the',
 'mine',
 'they',
 'soon',
 'find']

In [23]:
tokens[len_0:len_0 + 1]

['to']

In [24]:
train_len = len_0 + 1

text_sequences = []

for i in range(train_len, len(tokens)):
    # Construct sequence.
    seq = tokens[i - train_len: i]
    # Append.
    text_sequences.append(seq)

In [25]:
' '.join(text_sequences[0])

'five friends explore a supposedly haunted mine to celebrate halloween exactly one hundred years after a family was murdered in the mine they soon find to'

In [26]:
len(text_sequences[0])

26

In [27]:
for i in range(0, 5):
    print(' '.join(text_sequences[i]))
    print('-----')

five friends explore a supposedly haunted mine to celebrate halloween exactly one hundred years after a family was murdered in the mine they soon find to
-----
friends explore a supposedly haunted mine to celebrate halloween exactly one hundred years after a family was murdered in the mine they soon find to their
-----
explore a supposedly haunted mine to celebrate halloween exactly one hundred years after a family was murdered in the mine they soon find to their horror
-----
a supposedly haunted mine to celebrate halloween exactly one hundred years after a family was murdered in the mine they soon find to their horror that
-----
supposedly haunted mine to celebrate halloween exactly one hundred years after a family was murdered in the mine they soon find to their horror that the
-----


In [28]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

tokenizer.fit_on_texts(text_sequences)

In [29]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [30]:
sequences[0]

[856,
 99,
 5545,
 4,
 1978,
 1516,
 1979,
 3,
 5544,
 5543,
 5542,
 58,
 2827,
 148,
 30,
 4,
 50,
 36,
 256,
 8,
 1,
 1979,
 17,
 215,
 52,
 3]

In [31]:
tokenizer.index_word[6]

'is'

In [32]:
vocabulary_size = len(tokenizer.word_counts)

vocabulary_size

5545

In [33]:
sequences = np.array(sequences)

In [34]:
sequences

array([[ 856,   99, 5545, ...,  215,   52,    3],
       [  99, 5545,    4, ...,   52,    3,   26],
       [5545,    4, 1978, ...,    3,   26, 1980],
       ...,
       [  22,    4, 2711, ...,    1,   74,    7],
       [   4, 2711, 1204, ...,   74,    7,    5],
       [2711, 1204,   20, ...,    7,    5,  176]])

In [35]:
sequences.shape

(38903, 26)

In [36]:
from tensorflow.keras.utils import to_categorical

# select all but last word indices.
X = sequences[:, :-1]
X

array([[ 856,   99, 5545, ...,   17,  215,   52],
       [  99, 5545,    4, ...,  215,   52,    3],
       [5545,    4, 1978, ...,   52,    3,   26],
       ...,
       [  22,    4, 2711, ...,   20,    1,   74],
       [   4, 2711, 1204, ...,    1,   74,    7],
       [2711, 1204,   20, ...,   74,    7,    5]])

In [37]:
X.shape

(38903, 25)

In [38]:
seq_len = X.shape[1]

In [39]:
y = sequences[:, -1]
y

array([   3,   26, 1980, ...,    7,    5,  176])

In [40]:
y = to_categorical(y, num_classes=(vocabulary_size + 1))
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [32]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Bidirectional, LayerNormalization

def create_model(vocabulary_size, seq_len):
    
    model = Sequential()
    
    model.add(Embedding(input_dim=vocabulary_size, 
                        output_dim=seq_len, 
                        input_length=seq_len))
    
    model.add(Bidirectional(LSTM(units=50, recurrent_dropout=0.1, return_sequences=True)))
    
    model.add(LayerNormalization())
    
    model.add(Bidirectional(LSTM(units=50, recurrent_dropout=0.1)))
    
    model.add(LayerNormalization())
    
    model.add(Dense(units=50, activation='relu'))

    model.add(Dense(units=vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', 
                  optimizer='adam', 
                  metrics=['accuracy'])
    
    model.summary()
    
    return model

In [33]:
model = create_model(vocabulary_size=(vocabulary_size + 1), seq_len=seq_len)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 25)            138650    
_________________________________________________________________
bidirectional (Bidirectional (None, 25, 100)           30400     
_________________________________________________________________
layer_normalization (LayerNo (None, 25, 100)           200       
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100)               60400     
_________________________________________________________________
layer_normalization_1 (Layer (None, 100)               200       
_________________________________________________________________
dense (Dense)                (None, 50)                5050      
_________________________________________________________________
dense_1 (Dense)              (None, 5546)              2

In [34]:
model.fit(x=X, y=y, batch_size=128, epochs=15, verbose=1)

Epoch 1/15
304/304 [==============================] - 65s 188ms/step - loss: 6.8558 - accuracy: 0.0656
Epoch 2/15
304/304 [==============================] - 57s 187ms/step - loss: 6.3113 - accuracy: 0.0873
Epoch 3/15
304/304 [==============================] - 58s 191ms/step - loss: 5.8930 - accuracy: 0.1138
Epoch 4/15
304/304 [==============================] - 57s 187ms/step - loss: 5.5517 - accuracy: 0.1348
Epoch 5/15
304/304 [==============================] - 57s 188ms/step - loss: 5.2660 - accuracy: 0.1538
Epoch 6/15
304/304 [==============================] - 57s 187ms/step - loss: 5.0101 - accuracy: 0.1706
Epoch 7/15
304/304 [==============================] - 57s 187ms/step - loss: 4.7747 - accuracy: 0.1871
Epoch 8/15
304/304 [==============================] - 57s 187ms/step - loss: 4.5444 - accuracy: 0.2049
Epoch 9/15
304/304 [==============================] - 57s 187ms/step - loss: 4.3051 - accuracy: 0.2226
Epoch 10/15
304/304 [==============================] - 57s 186ms/step - l

In [35]:
from pickle import dump

dump(tokenizer, open('tokenizer', 'wb'))

In [36]:
model.save('model.h5')

In [13]:
from keras.models import load_model

model = load_model('model.h5')

In [14]:
from keras.preprocessing.sequence import pad_sequences 

def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    # List to store the generated words. 
    output_text = []
    # Set seed_text as input_text. 
    input_text = seed_text
    
    for i in range(num_gen_words):
        # Encode input text. 
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        # Add if the input tesxt does not have length len_0.
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        # Do the prediction. Here we automatically choose the word with highest probability.
        predict_x = model.predict(pad_encoded) 
        pred_word_ind = np.argmax(predict_x,axis=1)
        # Convert from numeric to word.
        pred_word = tokenizer.index_word[pred_word_ind.item(0)]
        # Attach predicted word. 
        input_text += ' ' + pred_word
        # Append new word to the list. 
        output_text.append(pred_word)
        
    return ' '.join(output_text)

In [15]:
sample_text = horror_df.iloc[1][:380]
print(sample_text)

Following a family dispute, Janet moves out of the home she shares with her older sister, Lara and their single mother, Maddie. She moves into apartment 1303 on the thirteenth floor of a downtown Detroit apartment building. A 9-year-old neighbor, Emily, explains to Janet that a previous occupant of her new apartment killed herself. Strange things begin to occur in the apartment


seed_text = sample_text[:190]
print(seed_text)

In [16]:
seed_text = sample_text[:213]
#seed_text = "Kael goes to the store and buys some groceries, but then encounters a wild man who "
print(seed_text)

Following a family dispute, Janet moves out of the home she shares with her older sister, Lara and their single mother, Maddie. She moves into apartment 1303 on the thirteenth floor of a downtown Detroit apartment


In [41]:
generated_text = generate_text(model=model, 
                               tokenizer=tokenizer,
                               seq_len=seq_len, 
                               seed_text=seed_text, 
                               num_gen_words=90)

print(seed_text + ' ' + generated_text + '...')

Following a family dispute, Janet moves out of the home she shares with her older sister, Lara and their single mother, Maddie. She moves into apartment 1303 on the thirteenth floor of a downtown Detroit apartment building which is a box of ashes in new eyes chris goes to youtube as the man secretly lambert realize that he wants to control and disgusts humanoid human full of human teller 's descendents continues to hunt in the woods causing the other world which is told to be paid up and they find himself in a mass tied with a parent at the woods and abby grabs the film and throw breaking out on a trance which reads a brief cavern and kills him in a trance with...
